In [12]:
import pandas as pd
import re
from fuzzywuzzy import fuzz, process
import spacy

municipios = ["Apodaca", "Monterrey", "Escobedo", "San Nicolás", "Santiago", "San Pedro Garza García", "Santa Catarina", "Guadalupe", "Saltillo", "Monte Morelos"]
# Crear el DataFrame con el nombre de la columna 'SETT_NAME'
df_municipios = pd.DataFrame(municipios, columns=['SETT_NAME'])
# Cargar el DataFrame original con entidades
df = pd.read_csv("df_con_entidades_por_etiqueta_TEST.csv")

# Cargar el DataFrame con las colonias conocidas (nombre y geometría)
df_colonias = df_municipios

# Función para normalizar el prefijo y eliminar puntos
def normalizar_prefijos(entidad):
    entidad = entidad.lower()  # Ignorar mayúsculas/minúsculas
    entidad = entidad.replace(".", "")  # Eliminar puntos

    # Reemplazar prefijos solo si están al inicio
    entidad = re.sub(r"^av( |$)", "avenida ", entidad)  # Reemplaza 'av' o 'ave' al inicio
    entidad = re.sub(r"^ave( |$)", "avenida ", entidad)
    entidad = re.sub(r"^col( |$)", "", entidad)  # Elimina el prefijo 'col' o 'colonia'
    entidad = re.sub(r"^colonia( |$)", "", entidad)
    entidad = re.sub(r"^municipio( |$)", "", entidad)
    entidad = re.sub(r"^mun( |$)", "", entidad)
    
    return entidad.strip()  # Eliminar espacios en blanco innecesarios

# Función para encontrar la mejor coincidencia y porcentaje
def encontrar_mejor_coincidencia(entidad, df_colonias):
    # Normalizar los nombres en la base de datos antes de comparar
    nombres_colonias_normalizados = [normalizar_prefijos(nombre) for nombre in df_colonias['SETT_NAME']]
    
    # Encontrar la mejor coincidencia y el puntaje
    mejor_coincidencia, score = process.extractOne(entidad, nombres_colonias_normalizados, scorer=fuzz.token_sort_ratio)
    
    # Encontrar el índice de la mejor coincidencia
    idx = nombres_colonias_normalizados.index(mejor_coincidencia)
    
    return mejor_coincidencia, score, idx

# Crear un nuevo DataFrame para almacenar las coincidencias válidas
df_actualizado = pd.DataFrame(columns=df.columns)

# Iterar sobre cada fila de la columna 'colonia'
for idx, colonia in df['municipio'].items():
    if pd.isna(colonia):  # Saltar filas vacías
        continue
    
    # Normalizar el nombre de la colonia antes de comparar
    colonia_normalizada = normalizar_prefijos(colonia)
    if colonia_normalizada == "spgg" or colonia_normalizada == "san pedro" or colonia_normalizada == "sp":
        colonia_normalizada = "San Pedro Garza García"
    elif colonia_normalizada == "mty":
        colonia_normalizada = "Monterrey"
    elif colonia_normalizada == "gpe":
        colonia_normalizada = "Guadalupe"
    # Encontrar la mejor coincidencia con la base de datos conocida
    mejor_coincidencia, score, idx_colonia = encontrar_mejor_coincidencia(colonia_normalizada, df_colonias)
    
    # Si la similitud es mayor al 75%, sobrescribir la colonia y agregar la geometría
    if score >= 60:
        df.loc[idx, 'municipio'] = mejor_coincidencia  # Sobrescribir la colonia
        df_actualizado = pd.concat([df_actualizado, df.loc[[idx]]])  # Añadir la fila actualizada
        print(f"Fila {idx}: Reemplazando '{colonia}' por '{mejor_coincidencia}' con {score}% de similitud")
    else:
        print(f"Fila {idx}: No se encontró coincidencia suficiente para '{colonia}' (Similitud: {score}%)")

# Guardar el DataFrame actualizado en un archivo CSV
df_actualizado.to_csv('df_con_colonias_actualizadas.csv', index=False)

print("DataFrame actualizado y guardado como 'df_con_colonias_actualizadas.csv'")


Fila 1: Reemplazando 'monterrey' por 'monterrey' con 100% de similitud
Fila 2: Reemplazando 'sannicolás' por 'san nicolás' con 63% de similitud
Fila 4: No se encontró coincidencia suficiente para 'marcoitnl' (Similitud: 35%)
Fila 9: No se encontró coincidencia suficiente para 'gto' (Similitud: 36%)
Fila 10: No se encontró coincidencia suficiente para 'veracruz' (Similitud: 35%)
Fila 13: No se encontró coincidencia suficiente para 'yucatán' (Similitud: 40%)
Fila 14: No se encontró coincidencia suficiente para 'veracruz' (Similitud: 35%)
Fila 15: No se encontró coincidencia suficiente para 'veracruz' (Similitud: 35%)
Fila 21: No se encontró coincidencia suficiente para 'gobiernomx' (Similitud: 44%)
Fila 22: No se encontró coincidencia suficiente para 'cdmx' (Similitud: 33%)
Fila 23: No se encontró coincidencia suficiente para 'cdmx' (Similitud: 33%)
Fila 24: No se encontró coincidencia suficiente para 'cutzamala' (Similitud: 44%)
Fila 25: No se encontró coincidencia suficiente para 'cutz